## Задача 1

In [5]:
import pandas as pd

file_path = "задание на аналитика.xlsx"

report1 = pd.read_excel(file_path, sheet_name="отчет 1")
report2 = pd.read_excel(file_path, sheet_name="отчет 2")
cession = pd.read_excel(file_path, sheet_name="цессия")

registry = report1.merge(
    report2,
    left_on="Номер контракта",
    right_on="NumContract",
    how="left"
)


print(registry)

     №  Номер контракта дата выдачи  Сумма выдачи  Задолженность по ОД  \
0    1             3547  2023-03-23         60000             47104.16   
1    2               68  2022-09-30         35000             11447.66   
2    3              389  2022-10-28         30000              4410.20   
3    4             1393  2022-12-20         40000             11088.24   
4    5             2050  2023-01-26         50000             42219.24   
5    6             2154  2023-01-31         20000              6606.81   
6    7             2148  2023-01-31         30000              8796.44   
7    8             3211  2023-03-10         30000             25533.57   
8    9             3512  2023-03-21         30000             26037.23   
9   10             6081  2023-06-18          9500              9500.00   
10  11             7524  2023-07-05          9500              9500.00   
11  12            12728  2023-08-26          9900              8655.17   
12  13            23629  2023-11-07   

## Задача 2 

In [6]:
sold_contracts = cession[cession["IsOutgoing"] == "Y"]["ContractID"]
registry = registry[~registry["ContractID"].isin(sold_contracts)]
print(registry)


     №  Номер контракта дата выдачи  Сумма выдачи  Задолженность по ОД  \
0    1             3547  2023-03-23         60000             47104.16   
1    2               68  2022-09-30         35000             11447.66   
2    3              389  2022-10-28         30000              4410.20   
3    4             1393  2022-12-20         40000             11088.24   
4    5             2050  2023-01-26         50000             42219.24   
5    6             2154  2023-01-31         20000              6606.81   
6    7             2148  2023-01-31         30000              8796.44   
7    8             3211  2023-03-10         30000             25533.57   
8    9             3512  2023-03-21         30000             26037.23   
9   10             6081  2023-06-18          9500              9500.00   
10  11             7524  2023-07-05          9500              9500.00   
11  12            12728  2023-08-26          9900              8655.17   
20  21            70431  2024-06-22   

## Задача 3

In [7]:
pivot = registry.pivot_table(
    index="SubdivisionName",
    values="Сумма выдачи",
    aggfunc="sum"
)

print(pivot)

                 Сумма выдачи
SubdivisionName              
Астрахань               30000
Белгород                65000
Вологда                 30000
Курск                   20000
Онлайн                  44800
Пенза                   30000
Санкт-Петербург         50000
Электросталь           100000


## Задачи по SQL (4 - 6)

In [ ]:
import pandas as pd
import sqlite3

file_path = "задание на аналитика.xlsx"

report1 = pd.read_excel(file_path, sheet_name="отчет 1")
report2 = pd.read_excel(file_path, sheet_name="отчет 2")

conn = sqlite3.connect(":memory:")

report1.to_sql("report1", conn, index=False, if_exists="replace")
report2.to_sql("report2", conn, index=False, if_exists="replace")

# задание 4
query_4 = """
SELECT *
FROM report2
WHERE SubdivisionName = 'Онлайн'
  AND issueDateTimestamp BETWEEN '2024-01-01' AND '2024-12-31'
"""
result_4 = pd.read_sql(query_4, conn)

# задание 5 
query_5 = """
SELECT 
    Status,
    COUNT(*) AS client_count,
    SUM(LoanAmount) AS total_issue_sum
FROM report2
GROUP BY Status
"""
result_5 = pd.read_sql(query_5, conn)

# задание 6
query_6 = """
SELECT *
FROM report1 r1
LEFT JOIN report2 r2
    ON r1."Номер контракта" = r2.NumContract
"""
result_6 = pd.read_sql(query_6, conn)

result_4.head(), result_5.head(), result_6.head()


(   NumContract  ContractID  ClientID inner_lead_id SubdivisionName  \
 0        35084      104036    327624        291631          Онлайн   
 1        36943      107766    330366        302295          Онлайн   
 2        37142      108361    330727        303297          Онлайн   
 3        37392      109111    331276        303744          Онлайн   
 4        37654      109897    331792        306104          Онлайн   
 
     issueDateTimestamp    RepaymentPlanDate  LoanAmount     Status  \
 0  2024-01-04 12:01:11  2024-03-28 00:00:00        9900  COLLECTOR   
 1  2024-01-11 23:17:09  2024-04-04 00:00:00        9900       SOLD   
 2  2024-01-12 21:48:11  2024-04-05 00:00:00        8000     REPAID   
 3  2024-01-14 08:17:10  2024-04-07 00:00:00        9900       SOLD   
 4  2024-01-15 13:47:12  2024-04-08 00:00:00        9900  COLLECTOR   
 
             DateStatus SumLastPay  
 0  2024-10-03 13:14:49    2347.26  
 1  2024-07-22 13:05:19       2340  
 2  2024-03-08 22:01:33    5705.7